In [57]:
import xarray as xr 
from src.utils.id_powerline_endpoints import load_counties, load_states
import geopandas as gpd
from shapely import Point
import pandas as pd 

import numpy as np
from shapely.vectorized import contains

In [58]:

# df = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-wb13-6h-0p25deg-chunk-1.zarr-v2")
df = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-6h-64x33.zarr")

In [59]:
filtered_ds = df.sel(time=slice('2021-01-01', None))
names = list(filtered_ds.coords)
filtered_ds = filtered_ds.assign_coords(longitude=xr.where(filtered_ds.longitude > 180,
                                                             filtered_ds.longitude - 360,
                                                             filtered_ds.longitude))

counties = load_counties()
# Create a single geometry for the entire United States by merging the counties
us_geom = counties.unary_union

# Create a 2D grid of longitude and latitude coordinates from filtered_ds
# (filtered_ds dimensions: latitude then longitude)
lon_grid, lat_grid = np.meshgrid(filtered_ds.longitude, filtered_ds.latitude)

# Create a boolean mask where each grid point is inside the US geometry
mask = contains(us_geom, lon_grid, lat_grid)

margin_degrees = 10 / 69  # approx. 10 miles in degrees
buffered_us = us_geom.buffer(margin_degrees)

lon_grid, lat_grid = np.meshgrid(filtered_ds.longitude, filtered_ds.latitude)
mask = contains(buffered_us, lon_grid, lat_grid)

mask_da = xr.DataArray(mask, dims=("latitude", "longitude"),
                            coords={"latitude": filtered_ds.latitude, "longitude": filtered_ds.longitude})
filtered_ds = filtered_ds.where(mask_da, drop=True)
print(filtered_ds)

/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/1898769529.py:9: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  us_geom = counties.unary_union
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/1898769529.py:16: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask = contains(us_geom, lon_grid, lat_grid)


<xarray.Dataset> Size: 61MB
Dimensions:                                           (time: 1460,
                                                       longitude: 16,
                                                       latitude: 6, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 48B ...
  * level                                             (level) int64 104B 50 ....
  * time                                              (time) datetime64[ns] 12kB ...
  * longitude                                         (longitude) float64 128B ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 561kB dask.array<chunksize=(16, 16, 6), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 561kB dask.array<chunksize=(16, 16, 6), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 561kB

/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/1898769529.py:22: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask = contains(buffered_us, lon_grid, lat_grid)


In [60]:
# Aggregate weather data for each state: average values over all grid nodes
# falling within the state geometry. Fallback to interpolation (centroid) if no grid node falls inside.

# load_states() and filtered_ds are already defined.
states = load_states().to_crs("EPSG:4326")
states["centroid"] = states.geometry.centroid

state_weather = {}  # dictionary to hold aggregated/interpolated weather data

# Compute meshgrid from filtered_ds coordinates.
lon_vals = filtered_ds.longitude.values
lat_vals = filtered_ds.latitude.values
lon_mesh, lat_mesh = np.meshgrid(lon_vals, lat_vals)

for idx, state in states.iterrows():
    # Compute mask for grid nodes that lie inside the state's geometry
    mask_state = contains(state.geometry, lon_mesh, lat_mesh)
    
    if mask_state.any():
        # Create an xarray DataArray for the mask with the same dims as filtered_ds spatial coords.
        da_mask = xr.DataArray(mask_state, dims=("latitude", "longitude"),
                                 coords={"latitude": lat_vals, "longitude": lon_vals})
        # Subset filtered_ds to grid points inside the state
        state_subset = filtered_ds.where(da_mask, drop=True)
        # Aggregate (e.g., average) over latitude and longitude dimensions.
        state_data = state_subset.mean(dim=["latitude", "longitude"])
    else:
        # Fallback: use the centroid to look up the nearest grid cell.
        print(state["NAME"], "has no grid points inside its geometry. Using centroid for interpolation.")
        # centroid = state["centroid"]
        # nearest_lat = filtered_ds.latitude.sel(latitude=centroid.y, method="nearest")
        # nearest_lon = filtered_ds.longitude.sel(longitude=centroid.x, method="nearest")
        # state_data = filtered_ds.sel(latitude=nearest_lat, longitude=nearest_lon, method="nearest")
    
    state_weather[state["NAME"]] = state_data

# Optionally, inspect the aggregated/interpolated weather data for each state
for state_name, data in state_weather.items():
    print(f"Weather data for {state_name}:")
    print(data)

/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  states["centroid"] = states.geometry.centroid
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzw

Nebraska has no grid points inside its geometry. Using centroid for interpolation.
Washington has no grid points inside its geometry. Using centroid for interpolation.
Kentucky has no grid points inside its geometry. Using centroid for interpolation.


/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5

Alabama has no grid points inside its geometry. Using centroid for interpolation.
Pennsylvania has no grid points inside its geometry. Using centroid for interpolation.
Missouri has no grid points inside its geometry. Using centroid for interpolation.
North Carolina has no grid points inside its geometry. Using centroid for interpolation.
Oklahoma has no grid points inside its geometry. Using centroid for interpolation.


/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5

New York has no grid points inside its geometry. Using centroid for interpolation.
Indiana has no grid points inside its geometry. Using centroid for interpolation.
Idaho has no grid points inside its geometry. Using centroid for interpolation.


/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5rz5163pjzwk435m0000gn/T/ipykernel_52197/4096390236.py:17: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask_state = contains(state.geometry, lon_mesh, lat_mesh)
/var/folders/64/2wpm0m9j5

North Dakota has no grid points inside its geometry. Using centroid for interpolation.
Hawaii has no grid points inside its geometry. Using centroid for interpolation.
Delaware has no grid points inside its geometry. Using centroid for interpolation.
Rhode Island has no grid points inside its geometry. Using centroid for interpolation.
Virginia has no grid points inside its geometry. Using centroid for interpolation.
Wyoming has no grid points inside its geometry. Using centroid for interpolation.
Louisiana has no grid points inside its geometry. Using centroid for interpolation.
Massachusetts has no grid points inside its geometry. Using centroid for interpolation.
Florida has no grid points inside its geometry. Using centroid for interpolation.
United States Virgin Islands has no grid points inside its geometry. Using centroid for interpolation.
Connecticut has no grid points inside its geometry. Using centroid for interpolation.
New Jersey has no grid points inside its geometry. Usi

In [61]:
state_weather["Georgia"]

<xarray.Dataset> Size: 643kB
Dimensions:                                           (time: 1460, level: 13)
Coordinates:
  * level                                             (level) int64 104B 50 ....
  * time                                              (time) datetime64[ns] 12kB ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time) float32 6kB dask.array<chunksize=(16,), meta=np.ndarray>
    10m_v_component_of_wind                           (time) float32 6kB dask.array<chunksize=(16,), meta=np.ndarray>
    10m_wind_speed                                    (time) float32 6kB dask.array<chunksize=(16,), meta=np.ndarray>
    2m_temperature                                    (time) float32 6kB dask.array<chunksize=(16,), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             float32 4B dask.array<chunksize=(), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           float32 4B dask.array<chunksize=(), meta=np.ndarray>
    type_of_low_vegetation                            float32 4B dask.array<chunksize=(), meta=np.ndarray>
    u_component_of_wind                               (time, level) float32 76kB dask.array<chunksize=(16, 13), meta=np.ndarray>
    v_component_of_wind                               (time, level) float32 76kB dask.array<chunksize=(16, 13), meta=np.ndarray>
    vertical_velocity                                 (time, level) float32 76kB dask.array<chunksize=(16, 13), meta=np.ndarray>
    wind_speed                                        (time, level) float32 76kB dask.array<chunksize=(16, 13), meta=np.ndarray>

In [ ]:
state_weather["Georgia"].to_dataframe().reset_index()

In [5]:


# example.apply(lambda row: Point(row['lon'], row['lat']))
df = df.to_dataframe()
lon_lat_df = df.index.to_frame(index=False)[['lon', 'lat']].drop_duplicates()
lon_lat_df['lon_fixed'] = lon_lat_df['lon'].apply(lambda x: x - 360 if x > 180 else x)
lon_lat_df['geometry'] = lon_lat_df.apply(lambda row: Point(row['lon_fixed'], row['lat']), axis=1)
gdf_points = gpd.GeoDataFrame(lon_lat_df, geometry='geometry',crs='EPSG:4326' )
counties = load_counties().to_crs('EPSG:4326')
ax = gdf_points["geometry"].plot()
counties.plot(ax=ax)    

KeyboardInterrupt: 

In [ ]:

joined = gpd.sjoin(gdf_points, counties[['GEOID', 'geometry']], how='left', predicate='within')
lon_lat_to_geoid = joined.set_index(['lon', 'lat'])['GEOID']
df_index = df.index.to_frame()

# Map each (lon, lat) to GEOID
df_index['county_geoid'] = df_index.set_index(['lon', 'lat']).index.map(lon_lat_to_geoid)

In [ ]:
ax = counties.plot()
joined[~joined["GEOID"].isna()].plot(ax=ax, markersize=10, c="red")
ax.figure.set_size_inches(100,10)

ax.set_xlim(-180,-60)
ax.set_ylim(10,75)

In [ ]:
from scipy.spatial import cKDTree
import numpy as np
# Ensure df has a MultiIndex with levels ['lon', 'lat', 'time', 'level', 'batch']
df_index = df.index.to_frame(index=False)

# Map each (lon, lat) to GEOID
lon_lat_to_geoid = joined.set_index(['lon', 'lat'])['GEOID']
df_index['county_geoid'] = df_index[['lon', 'lat']].apply(
    lambda row: lon_lat_to_geoid.get((row['lon'], row['lat']), np.nan), axis=1
)

# Step 1: Compute county centroids
county_centroids = counties.geometry.centroid
counties['centroid'] = county_centroids

# Step 2: Extract latitude and longitude of centroids
centroid_coords = np.array([[point.x, point.y] for point in county_centroids])

# Step 3: Build a KDTree for nearest neighbor search
lon_lat_coords = lon_lat_df[['lon_fixed', 'lat']].to_numpy()
tree = cKDTree(lon_lat_coords)
# df.columns = Index(['geopotential_at_surface', 'land_sea_mask', '2m_temperature',
    #    'mean_sea_level_pressure', '10m_v_component_of_wind',
    #    '10m_u_component_of_wind', 'total_precipitation_6hr',
    #    'toa_incident_solar_radiation', 'temperature', 'geopotential',
    #    'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
    #    'specific_humidity', 'datetime'],
    #   dtype='object')
# df.index = MultiIndex([(  0.0, -90.0,  '0 days 00:00:00',    1, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',    2, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',    3, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',    5, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',    7, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',   10, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',   20, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',   30, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',   50, 0),
        #     (  0.0, -90.0,  '0 days 00:00:00',   70, 0),
        #     ...
        #     (359.0,  90.0, '10 days 06:00:00',  775, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  800, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  825, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  850, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  875, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  900, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  925, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  950, 0),
        #     (359.0,  90.0, '10 days 06:00:00',  975, 0),
        #     (359.0,  90.0, '10 days 06:00:00', 1000, 0)],
        #    names=['lon', 'lat', 'time', 'level', 'batch'], length=101258640)
# Find the 4 nearest neighbors for each centroid
# Find the 4 nearest neighbors for each centroid
distances, indices = tree.query(centroid_coords, k=4)

In [ ]:
import tqdm
# Perform bilinear interpolation with proper handling of MultiIndex
county_data_list = []
keep  =['geopotential_at_surface', 'land_sea_mask', '2m_temperature',
       'mean_sea_level_pressure', '10m_v_component_of_wind',
       '10m_u_component_of_wind', 'total_precipitation_6hr',
       'toa_incident_solar_radiation', 'temperature', 'geopotential',
       'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
       'specific_humidity']
for i, (dists, nbr_idxs) in tqdm.tqdm(enumerate(zip(distances, indices)), total=len(distances)):
    county_id = counties.iloc[i]['GEOID']
    
    # Compute weights for the 4 neighbors
    weights = 1 / (dists + 1e-6)
    weights /= weights.sum()
    
    # Extract weather data for the 4 neighbors
    nbr_dfs = []
    for j in range(4):
        coord = lon_lat_df.iloc[nbr_idxs[j]]
        lon_val, lat_val = coord['lon'], coord['lat']
        try:
            nbr_df = df.xs((lon_val, lat_val), level=['lon', 'lat'])
        except KeyError:
            print("KeyError: ", lon_val, lat_val)
            # Handle missing data by creating a dummy DataFrame
            dummy_index = df.index.levels[2]  # Assuming 'time' is the 3rd level
            nbr_df = pd.DataFrame(np.nan, index=dummy_index, columns=df.columns)
        nbr_dfs.append(nbr_df)
    
    # Weighted sum for bilinear interpolation
    interp_df = nbr_dfs[0][keep] * weights[0]
    # print(interp_df)
    for w, nbr in zip(weights[1:], nbr_dfs[1:]):
        interp_df += nbr[keep] * w
    
    # Add county information and reset index
    interp_df = interp_df.reset_index()
    interp_df['county'] = county_id
    interp_df.set_index(['county', 'datetime', 'level', 'batch'], inplace=True)
    # print(interp_df)
    county_data_list.append(interp_df)

In [ ]:

# Concatenate all county DataFrames
interpolated_df = pd.concat(county_data_list)
interpolated_df.to_parquet("../data/county_data2",compression="gzip", engine="pyarrow", index=True, partition_cols=["county"])